In [1]:
from dgfip.read import get_iris, get_structures
import numpy as np

In [2]:
structures = get_structures()
centres = structures[structures['TYPE DE SERVICE']=='Centre de Finances publiques']

In [7]:
import geopandas as gpd
# fond de carte des départements
dep = gpd.read_file('https://github.com/gregoiredavid/france-geojson/raw/master/departements-version-simplifiee.geojson', crs='4236')

In [5]:
x= np.array(centres.to_crs(crs='2154').geometry.x)
y= np.array(centres.to_crs(crs='2154').geometry.y)
points = np.stack((x,y), axis=-1)

In [8]:
from shapely.geometry import box
from libpysal.cg.voronoi  import voronoi, voronoi_frames
points_u, index = np.unique(points, axis=0, return_index=True)
region_df, point_df = voronoi_frames(points_u, clip = box(*dep.to_crs('2154').total_bounds))

In [58]:
areas = region_df.set_crs("2154").clip(dep.to_crs('2154')).area/10**6

In [59]:
region_df['area'] = areas

In [17]:
centres_u = centres.iloc[index]

In [22]:
iris = get_iris(compute_distance=False)

In [38]:
region_df['id'] = region_df.index

In [60]:
inter = region_df.set_crs('2154').sjoin(iris)

In [61]:
gr = inter.groupby('id').sum()

In [62]:
geo = gpd.GeoDataFrame(gr, geometry= region_df.geometry)

In [1]:
from dgfip.influence import create_influence
create_influence(save=True)

,index_right,MODIF_IRIS,P17_POP,P17_POP0002,P17_POP0305,P17_POP0610,P17_POP1117,P17_POP1824,P17_POP2539,P17_POP4054,...,C17_F15P_CS6,C17_F15P_CS7,C17_F15P_CS8,P17_POP_FR,P17_POP_ETR,P17_POP_IMM,P17_PMEN,P17_PHORMEN,geometry,area
id,,,,,,,,,,,,,,,,,,,,,
0,241993,0,20701.000000,429.320510,512.373807,974.637768,1555.458422,1092.528391,2570.277823,3763.858534,...,287.268548,3903.732750,1538.369795,20309.514670,391.485330,528.635869,19909.715399,791.284601,"MULTIPOLYGON (((-4.54721 48.31680, -4.55269 48...",194.403579
1,2766966,0,269378.432157,8387.950732,9100.079411,15691.614298,22984.048851,31167.539628,49407.973431,50994.511825,...,4603.010368,32427.671800,24869.693062,260517.503377,8860.928780,11547.705721,261616.964707,7761.467450,"MULTIPOLYGON (((-4.62814 48.33753, -4.67843 48...",663.009353
2,968173,0,88994.682629,2484.927089,2908.125655,4978.403308,7751.069360,9329.671127,16341.426736,17626.957026,...,1107.335031,10467.407311,7686.115760,86680.708189,2313.974439,3263.758292,86510.387615,2484.295013,"MULTIPOLYGON (((-4.57955 48.31723, -4.58052 48...",79.196665
3,947980,0,60477.000000,1450.964793,1651.049158,3245.423295,4815.412723,2760.174677,7936.505467,12163.463270,...,1595.458653,11018.753678,3537.916866,59632.038756,844.961244,1211.265632,59270.721656,1206.278344,"POLYGON ((-4.16497 48.10791, -4.25447 47.97889...",455.928247
4,500950,0,65644.000000,1543.339189,1823.892550,3445.670477,5394.022824,3135.642128,8600.938952,13251.552283,...,1271.852634,11719.589911,4011.759084,64954.583417,689.416583,1186.868920,64339.434999,1304.565001,"POLYGON ((-4.25447 47.97889, -4.13460 47.92896...",305.746812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,1787301,0,19222.000000,395.004423,395.259305,619.171437,961.040791,3144.939890,2843.213655,3240.576834,...,197.561834,2412.923655,3000.761874,17603.552477,1618.447523,1666.156810,18112.287523,1109.712477,"POLYGON ((9.06054 42.05607, 8.74999 42.24223, ...",1032.424184
1308,482220,0,27058.000000,814.568046,853.859280,1691.398671,2183.459965,1518.173281,4746.944430,6007.705913,...,301.634764,2828.824291,3043.609867,23270.196179,3787.803821,4021.081326,26869.353797,188.646203,"POLYGON ((9.40108 41.78247, 9.40804 41.76596, ...",757.294748
1309,1121739,0,79548.000000,2237.150788,2508.597645,4590.135727,6088.399040,4995.500802,14702.826744,16598.055142,...,725.687826,8978.666568,10081.802146,73376.240431,6171.759569,6919.082397,78372.753287,1175.246713,"POLYGON ((9.17421 42.73407, 9.19806 42.72498, ...",554.393671
